In [32]:
import itertools
import numpy as np

def indisBul(D):
    S = np.sort(D)
    M = np.zeros((len(D), 2))
    
    for i in range(len(D)):
        M[i, 0] = D[i]
        indx = np.where(S == D[i])[0]
        
        for j in range(len(indx)):
            if np.any(M[:, 1] == indx[j]):
                continue
            else:
                M[i, 1] = indx[j]
                break
                
    return M

def motifyeni(data, Pboyut):
    S = np.array(data)
    PB = Pboyut  # Pencere boyu
    P = list(itertools.permutations(range(1, PB + 1)))  # pencere boyuna bağlı tüm permütasyonlar
    P = np.array(P)
    #print(P)
    Pc = np.zeros(P.shape[0])
    
    for i in range(len(S) - PB + 1):
        a = S[i:i + PB]  # pencere boyutu kadar sinyalden alınan parça
        #print(a)
        b = indisBul(a)  # alınan parçanın örüntüsünün elde edilmesi
        #print(b)
        b=b+1
        # hangi patterne uyduğu bulunuyor
        patindx = np.where(np.sum(np.abs(P - np.tile(b[:, 1], (P.shape[0], 1))), axis=1) == 0)[0]
        Pc[patindx] += 1  # bulunan patter sayacı 1 arttırılıyor

    motifs = Pc
    return motifs

# Example usage:
data = [10, 24, 33, 41, 5, 6, 7, 8, 9,5]
Pboyut = 3
result = motifyeni(data, Pboyut)
print(result)




[5. 0. 0. 2. 1. 0.]


In [40]:
import numpy as np


# Read data from files
with open('spam/D1.txt', 'r') as file:
    A = file.readlines()

with open('spam/D2.txt', 'r') as file:
    B = file.readlines()

features = []
PB = 5

for i in range(len(A)):
    dosya = A[i].strip()
    bytes = [ord(char) for char in dosya]

    if len(bytes) > 10:
        veri = bytes
        a = motifyeni(veri, PB)

        if B[i].strip() != 'ham':
            features.append(np.concatenate((a, [0])))
        else:
            features.append(np.concatenate((a, [1])))

# features array now contains the extracted features and labels
features = np.array(features)
print(features)
print(features.shape)

[[0. 2. 1. ... 2. 4. 0.]
 [2. 3. 2. ... 1. 3. 0.]
 [4. 3. 1. ... 1. 4. 0.]
 ...
 [4. 0. 0. ... 0. 0. 0.]
 [3. 1. 2. ... 2. 3. 0.]
 [1. 0. 5. ... 1. 3. 0.]]
(1233, 121)


In [41]:

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Giriş öznitelikleri ve çıkış sütununu ayırma
X = features[:, :features.shape[1]-1]  
y = features[:, features.shape[1]-1]   # Son sütun çıkış


# Veriyi eğitim ve test setlerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# KNN sınıflandırıcı
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print("KNN Başarı:", knn_accuracy)

knn_conf_matrix = confusion_matrix(y_test, knn_predictions)
print("KNN Karışıklık Matrisi:")
print(knn_conf_matrix)

# Naive Bayes sınıflandırıcı
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naive Bayes Başarı", nb_accuracy)

nb_conf_matrix = confusion_matrix(y_test, nb_predictions)
print("Naive Bayes Karışıklık Matrisi:")
print(nb_conf_matrix)

KNN Başarı: 0.9378378378378378
KNN Karışıklık Matrisi:
[[ 62  22]
 [  1 285]]
Naive Bayes Başarı 0.8594594594594595
Naive Bayes Karışıklık Matrisi:
[[ 78   6]
 [ 46 240]]
